In [ ]:
# !mkdir -p images

In [ ]:
# !mv train2019 images/
# !mv val2019 images/
# !mv test2019 images/

In [ ]:
# !rm -rf labels/train2019/
# !rm -rf labels/val2019/
# !rm -rf labels/test2019/
!mkdir -p labels/train2019
!mkdir -p labels/val2019
!mkdir -p labels/test2019

In [37]:
import os
import json
import shutil
from tqdm import tqdm

In [ ]:
train = json.load(open('instances_train2019.json', 'r'))
val = json.load(open('instances_val2019.json', 'r'))
test = json.load(open('instances_test2019.json', 'r'))

In [ ]:
list(train.keys())

In [ ]:
print(len(train['categories']))
print(train['categories'][:2])

In [ ]:
classes = []
for category in train['categories']:
    classes.append(category['name'])

In [ ]:
print(len(train['__raw_Chinese_name_df']))
print(train['__raw_Chinese_name_df'][:2])

In [ ]:
print(len(train['images']))
print(train['images'][:2])

In [ ]:
print(len(train['annotations']))
print(train['annotations'][:2])

In [ ]:
!mkdir -p crop_images/train2019
!mkdir -p crop_images/val2019
!mkdir -p crop_images/test2019

In [ ]:
train_images = {}

for image in tqdm(train['images']):
    image_id = image['id']
    train_images[image_id] = image
    
for annotation in tqdm(train['annotations']):
    image_id = annotation['image_id']
    image = train_images[image_id]
    
    file_name = image['file_name']
    width = image['width']
    height = image['height']
    
    bbox = annotation['bbox']
    x = (bbox[0]+bbox[2]/2)/width
    y = (bbox[1]+bbox[3]/2)/height
    w = bbox[2]/width
    h = bbox[3]/height
    category_id = annotation['category_id']-1
    
    with open('labels/train2019/'+file_name.replace('jpg', 'txt'), 'a') as fout:
        fout.write(str(category_id)+' '+str(x)+' '+str(y)+' '+str(w)+' '+str(h)+'\n')

In [ ]:
print(len(val['images']))
print(val['images'][:2])
print(len(val['annotations']))
print(val['annotations'][:2])

In [ ]:
val_images = {}

for image in tqdm(val['images']):
    image_id = image['id']
    val_images[image_id] = image
    
for annotation in tqdm(val['annotations']):
    image_id = annotation['image_id']
    image = val_images[image_id]
    
    file_name = image['file_name']
    width = image['width']
    height = image['height']
    
    bbox = annotation['bbox']
    x = (bbox[0]+bbox[2]/2)/width
    y = (bbox[1]+bbox[3]/2)/height
    w = bbox[2]/width
    h = bbox[3]/height
    category_id = annotation['category_id']-1
    
    with open('labels/val2019/'+file_name.replace('jpg', 'txt'), 'a') as fout:
        fout.write(str(category_id)+' '+str(x)+' '+str(y)+' '+str(w)+' '+str(h)+'\n')

In [ ]:
print(len(test['images']))
print(test['images'][:2])
print(len(test['annotations']))
print(test['annotations'][:2])

In [ ]:
test_images = {}

for image in tqdm(test['images']):
    image_id = image['id']
    test_images[image_id] = image
    
for annotation in tqdm(test['annotations']):
    image_id = annotation['image_id']
    image = test_images[image_id]
    
    file_name = image['file_name']
    width = image['width']
    height = image['height']
    
    bbox = annotation['bbox']
    x = (bbox[0]+bbox[2]/2)/width
    y = (bbox[1]+bbox[3]/2)/height
    w = bbox[2]/width
    h = bbox[3]/height
    category_id = annotation['category_id']-1
    
    with open('labels/test2019/'+file_name.replace('jpg', 'txt'), 'a') as fout:
        fout.write(str(category_id)+' '+str(x)+' '+str(y)+' '+str(w)+' '+str(h)+'\n')

In [ ]:
!mkdir cfg

In [ ]:
with open('cfg/rpc.yaml', 'w') as fout:
    fout.write('path: /home/ec2-user/SageMaker/data/retail_product_checkout/\n')
#     fout.write('train: images/train2019\n')
#     fout.write('val: images/val2019\n')
#     fout.write('test: images/test2019\n')
    fout.write('train: \n')
    fout.write('  - images/train2019\n')
    fout.write('  - images/val2019\n')
    fout.write('val: images/test2019\n')
    fout.write('nc: '+str(len(train['categories']))+'\n')
    fout.write('classes: '+str(classes)+'\n')

## Crop images

In [ ]:
import cv2

In [ ]:
train_labels = os.listdir('labels/train2019/')
val_labels = os.listdir('labels/val2019/')
test_labels = os.listdir('labels/test2019/')
print(len(train_labels), len(val_labels), len(test_labels))

In [ ]:
for label in tqdm(train_labels):
    if label.endswith('txt'):
        label_filename = os.path.join('labels/train2019/', label)
        image_filename = os.path.join('images/train2019/', label.replace('txt', 'jpg'))
#         print('label_filename:', label_filename)
#         print('image_filename:', image_filename)
        image = cv2.imread(image_filename)
        width = image.shape[1]
        height = image.shape[0]
        with open(label_filename, 'r') as fin:
            lines = [line.rstrip('\n') for line in fin]
            for i, line in enumerate(lines):
#                 line = line.strip()
                params = line.split(' ')
                class_id = int(params[0])
                center_x = float(params[1])
                center_y = float(params[2])
                w = float(params[3])
                h = float(params[4])
                xmin = int((center_x-w/2)*width)
                xmax = int((center_x+w/2)*width)
                ymin = int((center_y-h/2)*height)
                ymax = int((center_y+h/2)*height)
                sub_image = image[ymin:ymax, xmin:xmax, :]
                cv2.imwrite(image_filename.replace('images', 'crop_images').replace('.jpg', '_'+str(i)+'_'+str(class_id)+'_'+str(train['__raw_Chinese_name_df'][class_id]['name'])+'.jpg'), sub_image)        

In [ ]:
for label in tqdm(val_labels):
    if label.endswith('txt'):
        label_filename = os.path.join('labels/val2019/', label)
        image_filename = os.path.join('images/val2019/', label.replace('txt', 'jpg'))
#         print('label_filename:', label_filename)
#         print('image_filename:', image_filename)
        image = cv2.imread(image_filename)
        width = image.shape[1]
        height = image.shape[0]
        with open(label_filename, 'r') as fin:
            lines = [line.rstrip('\n') for line in fin]
            for i, line in enumerate(lines):
#                 line = line.strip()
                params = line.split(' ')
                class_id = int(params[0])
                center_x = float(params[1])
                center_y = float(params[2])
                w = float(params[3])
                h = float(params[4])
                xmin = int((center_x-w/2)*width)
                xmax = int((center_x+w/2)*width)
                ymin = int((center_y-h/2)*height)
                ymax = int((center_y+h/2)*height)
                sub_image = image[ymin:ymax, xmin:xmax, :]
                cv2.imwrite(image_filename.replace('images', 'crop_images').replace('.jpg', '_'+str(i)+'_'+str(class_id)+'_'+str(train['__raw_Chinese_name_df'][class_id]['name'])+'.jpg'), sub_image)        

In [ ]:
for label in tqdm(test_labels):
    if label.endswith('txt'):
        label_filename = os.path.join('labels/test2019/', label)
        image_filename = os.path.join('images/test2019/', label.replace('txt', 'jpg'))
#         print('label_filename:', label_filename)
#         print('image_filename:', image_filename)
        image = cv2.imread(image_filename)
        width = image.shape[1]
        height = image.shape[0]
        with open(label_filename, 'r') as fin:
            lines = [line.rstrip('\n') for line in fin]
            for i, line in enumerate(lines):
#                 line = line.strip()
                params = line.split(' ')
                class_id = int(params[0])
                center_x = float(params[1])
                center_y = float(params[2])
                w = float(params[3])
                h = float(params[4])
                xmin = int((center_x-w/2)*width)
                xmax = int((center_x+w/2)*width)
                ymin = int((center_y-h/2)*height)
                ymax = int((center_y+h/2)*height)
                sub_image = image[ymin:ymax, xmin:xmax, :]
                cv2.imwrite(image_filename.replace('images', 'crop_images').replace('.jpg', '_'+str(i)+'_'+str(class_id)+'_'+str(train['__raw_Chinese_name_df'][class_id]['name'])+'.jpg'), sub_image)        

In [33]:
for df in train['__raw_Chinese_name_df']:
    name = df['name']
    os.makedirs(os.path.join('crop_images', 'train2019', name))
    os.makedirs(os.path.join('crop_images', 'val2019', name))
    os.makedirs(os.path.join('crop_images', 'test2019', name))

In [34]:
train_crop_images = os.listdir(os.path.join('crop_images', 'train2019'))
val_crop_images = os.listdir(os.path.join('crop_images', 'val2019'))
test_crop_images = os.listdir(os.path.join('crop_images', 'test2019'))
print(len(train_crop_images), len(val_crop_images), len(test_crop_images))

53940 73803 294533


In [41]:
for image in tqdm(train_crop_images):
    if image.endswith('jpg'):
        dir_name = image[:-4].split('_')[-1]
        shutil.move(os.path.join('crop_images', 'train2019', image), os.path.join('crop_images', 'train2019', dir_name))

100%|██████████| 53940/53940 [00:01<00:00, 36480.78it/s]


In [42]:
for image in tqdm(val_crop_images):
    if image.endswith('jpg'):
        dir_name = image[:-4].split('_')[-1]
        shutil.move(os.path.join('crop_images', 'val2019', image), os.path.join('crop_images', 'val2019', dir_name))

100%|██████████| 73803/73803 [00:01<00:00, 42463.22it/s]


In [43]:
for image in tqdm(test_crop_images):
    if image.endswith('jpg'):
        dir_name = image[:-4].split('_')[-1]
        shutil.move(os.path.join('crop_images', 'test2019', image), os.path.join('crop_images', 'test2019', dir_name))

100%|██████████| 294533/294533 [00:07<00:00, 38325.47it/s]
